In [1]:
#basic package
import tqdm
import os
import numpy as np
import glob
import pandas as pd
import sys
import time
import datetime as dt
import itertools
import re
import math
from collections import Counter
import operator

#scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#clustering
from sklearn.cluster import KMeans #only numerical var
from sklearn.metrics import silhouette_samples, silhouette_score
import kmodes
from kmodes.kmodes import KModes #with categorical var as well

#plot
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import preprocessing_Origins,verification_based_on_initial_record,\
time_series_henColumn_tsRow,is_day,vertical_travel_distance, li_missingZone_mvtPerc_DU, li_event_chaoticmvt_z_d,\
ZoneVariable, HenDailyVariable_Origins, boxplot_distribution_entropy,\
nbr_transition, max_duration_zones, openDevice, kmeans_clustering
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


# Define parameters

In [3]:
print('change the configuration file if not done yet!')
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
dico_matching = config.dico_matching
print(id_run)
path_extracted_data_visual = os.path.join(path_extracted_data,'visual')
#create a director if not existing
if not os.path.exists(path_extracted_data_visual):
    os.makedirs(path_extracted_data_visual)

change the configuration file if not done yet!
correctlightschedule_


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Variables at Hen level

In [4]:
#WARNING: we do it at seconds level, so the miliseconds from the general cleaning is not used!! 

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
#for being more reproductible, we open the file that was saved from cleaning
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_CLEANEDDATA.csv'), sep=';', parse_dates=['Timestamp', 'date']) 
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1323363, 16)


,HenID,PenID,system,Zone,model_prediction,Timestamp,date,next_record_date,duration,previous_record_date,previous_duration,next_zone,previous_zone,previous_previous_zone,correction_is_consecutive_equal_initial_zone,is_WG_open
0,hen_71,pen12,10 - 12,3_Zone,1.0,2020-09-29 09:07:00,2020-09-29,2020-09-29 09:08:26.000,0 days 00:01:26,NaN,NaN,3_Zone,NaN,NaN,False,False
1,hen_7,pen11,10 - 12,3_Zone,1.0,2020-09-29 09:08:12,2020-09-29,2020-09-29 09:12:16.000,0 days 00:04:04,NaN,NaN,3_Zone,NaN,NaN,False,False
2,hen_10,pen11,10 - 12,3_Zone,1.0,2020-09-29 09:19:19,2020-09-29,2020-09-29 09:20:27.000,0 days 00:01:08,NaN,NaN,3_Zone,NaN,NaN,False,False


In [6]:
#open time series per session and compute the variables for each session (car time series make sence at session level), et en 
#plus des variables tel que running entropy over the whole session ateach last timestamp of each level make sence only at 
#session level
#once to much data, do this per pen!!! with name=pens!!
START_TIME = time.perf_counter()
for p in tqdm.tqdm(df['PenID'].unique()):
    print(p)
    df_daily = HenDailyVariable_Origins(df[df['PenID']==p], config, name_=p, timestamp_name='Timestamp', 
                                        has_cons_equal_zone=False)
    print(df_daily.shape)
    df_daily.head(3)
END_TIME = time.perf_counter()
print ("Total running time: %.2f mn" %((END_TIME-START_TIME)/60))  

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

pen12


C:\Users\camil\Desktop\animals_code\PhD\UTILS.py:3933: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values([timestamp_name], inplace=True)
C:\Users\camil\Desktop\animals_code\PhD\UTILS.py:3935: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[timestamp_name] = df[timestamp_name].map(lambda x: dt.datetime(x.year,x.month,x.day,x.hour,x.minute, x.second))


----------------- Create time serie
in this time series there is 29 hens
The initial starting date in over all is: 2020-09-29 09:07:00, and the ending date will be: 2021-03-01 08:32:40
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-09-29 09:07:00, and the ending date will be: 2021-03-01 23:59:59
Total running time: 2.68 mn
your time series has 1 seconds between two timestamps
----------------- main night zone and nbr of transitions over night....
----------------- total duration per Zone in seconds....
----------------- first timestamp in each zone per day....
----------------- number of Zone (excluding nan)....
----------------- compute some variables based on a list of zones over a day....
----------------NESTBOX



100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:09<00:00,  3.14it/s]
C:\Users\camil\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1366: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


...

-------------- Lets remove dates of tags when they were having more or equal than 10 times LFCOUNTER=0 a day


-------------- Lets remove the 30.09.2020 for the tags taht still had no transition from before the light went on 


save


100%|████████████████████████████████████████████████████████████████████████████████| 8/8 [3:26:51<00:00, 1551.41s/it]

Total running time: 28.97 mn
(3303, 134)
Total running time: 206.86 mn


In [7]:
#Concatenate all HenVariables csv into one
li_paths_var = glob.glob(os.path.join(path_extracted_data, id_run+'_daily__pen'+'*_variables.csv'))
li_df = []
if len(li_paths_var)!=8:
    print('ERROR: not the correct number of files, there must have one per pen')
    sys.exit()
for path_var in tqdm.tqdm(li_paths_var):
    #for being more reproductible, we open the file that was saved from cleaning
    df_ = pd.read_csv(path_var, sep=';',parse_dates=['level', 'FirstTimestamp_1_Zone', 'FirstTimestamp_2_Zone', 
                                                     'FirstTimestamp_3_Zone','FirstTimestamp_4_Zone',
                                                     'FirstTimestamp_5_Zone']) 
    print(df_.shape)
    li_df.append(df_)
df_daily = pd.concat(li_df)
df_daily.to_csv(os.path.join(path_extracted_data, id_run+'_daily_ALL_variables.csv'), sep=';', index=False)
print(df_daily.shape)
df_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
 12%|██████████▌                                                                         | 1/8 [00:04<00:28,  4.13s/it]

(3487, 134)


 25%|█████████████████████                                                               | 2/8 [00:10<00:27,  4.66s/it]

(3493, 134)


 38%|███████████████████████████████▌                                                    | 3/8 [00:14<00:22,  4.56s/it]

...

(3303, 134)


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:33<00:00,  4.16s/it]

(3398, 134)


(27593, 134)


,HenID,level,duration_1_Zone,duration_2_Zone,duration_3_Zone,duration_4_Zone,duration_5_Zone,verification_daily_total_duration,dur_values,dur_values_normalized,...,list_of_night20_2_temperature,nbr_temperature_nnnan,temperature_night20_2_median,temperature_night20_2_max,temperature_night20_2_var,list_of_MovementCounter_day,len_MovementCounter_day,MovementCounter_day_amount_nnactivity,MovementCounter_day_max,MovementCounter_day_mean
0,hen_130,2020-10-04,0.0,140.0,1166.0,79.0,30613.0,31998.0,"[0.0, 140.0, 1166.0, 79.0, 30613.0]","[0.0, 0.004375273454590912, 0.0364397774860928...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,hen_130,2020-10-05,0.0,0.0,0.0,0.0,32400.0,32400.0,"[0.0, 0.0, 0.0, 0.0, 32400.0]","[0.0, 0.0, 0.0, 0.0, 1.0]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,hen_130,2020-10-06,0.0,0.0,780.0,325.0,31295.0,32400.0,"[0.0, 0.0, 780.0, 325.0, 31295.0]","[0.0, 0.0, 0.024074074074074074, 0.01003086419...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Chi2distance

#computed here as its a variable and not a visual computed out of variable
#compute for every level a symmetric chi2distance heatmap
if compute_chi2_distance:   

    #sort by lowest entropy (i.e. need less info to predict futur. more predictibale should induce more similarities 
    #(as less different solution)), to potentialy make a nicer visual
    df_ = df_daily.groupby(['HenID'])['distribution_entropy'].agg(lambda x:np.mean(x)).reset_index().sort_values(['distribution_entropy'])
    li = df_['HenID'].tolist()
    axis_label = [i.split('_')[1] for i in li]
    #create path to save visual if not existing
    path_ = os.path.join(path_extracted_data,'visual','chi2distance')
    #create a director if not existing
    if not os.path.exists(path_):
        os.makedirs(path_)

    print('----------------- Compute Chi2 distance....')
    for d in li_date:
        M = np.zeros(shape=(len(li),len(li)))
        for i, h1 in enumerate(li[:-1]):
            for j in range(i+1,len(li)):
                h2 = li[j]
                li_hen_in = df_daily[df_daily['level']==d]['HenID'].unique()
                #if both hen have at least one record this day (typically not always all hen have values the first day of session)
                if (h1 in li_hen_in) & (h2 in li_hen_in):
                    l1 = df_daily[(df_daily['HenID']==h1)&(df_daily['level']==d)]['dur_values_normalized'].values[0]
                    l2 = df_daily[(df_daily['HenID']==h2)&(df_daily['level']==d)]['dur_values_normalized'].values[0]
                    chi2 = chi2_distance(l1,l2)
                    M[i][j] = chi2
                    M[j][i] = chi2
        #clear old plot
        plt.figure()
        sns.set(font_scale=0.25) 
        sns.heatmap(M, cmap="YlGnBu", xticklabels=axis_label, yticklabels=axis_label)
        plt.title(d)
        if save:
            plt.savefig(os.path.join(path_,id_run+'_chi2distance_'+str(d).split('T')[0]+'_daily_'+'.png'), dpi=300,
                        format='png',bbox_inches='tight')
        #plt.show() 
        plt.close()

